In [1]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "ViShaYa"
print("Let's chat! (type 'quit' to exit)")


Let's chat! (type 'quit' to exit)


In [2]:
sentence="Hello"
# if sentence == "quit":
#     break

sentence = tokenize(sentence)
X = bag_of_words(sentence, all_words)
X = X.reshape(1, X.shape[0])
X = torch.from_numpy(X).to(device)

output = model(X)
_, predicted = torch.max(output, dim=1)

tag = tags[predicted.item()]

probs = torch.softmax(output, dim=1)
prob = probs[0][predicted.item()]


In [3]:
tag

'greeting'

In [4]:
if prob.item() > 0.75:
    for intent in intents['intents']:
        if tag == intent["tag"]:
            print(f"{bot_name}: {random.choice(intent['responses'])}")
else:
    print(f"{bot_name}: I do not understand...")

ViShaYa: Hi there, how can I help?


In [5]:
import pandas as pd
from datetime import datetime, timedelta

# Load the CSV file
df = pd.read_csv('dummy_orders.csv')

def check_warranty(order_id):
    # Find the order in the dataframe
    order = df[df['OrderID'] == order_id]
    
    if order.empty:
        print(f"Order ID {order_id} not found.")
        return
    
    # Get the date of purchase
    date_of_purchase = datetime.strptime(order.iloc[0]['DateOfPurchase'], '%Y-%m-%d')
    
    # Calculate the expiry date
    expiry_date = date_of_purchase + timedelta(days=365)
    
    # Check if the warranty is still valid
    if datetime.now() <= expiry_date:
        print(f"The warranty for Order ID {order_id} is still valid.")
    else:
        print(f"The warranty for Order ID {order_id} has expired.")
    
    print(f"Expiry Date: {expiry_date.strftime('%Y-%m-%d')}")

# Get Order ID from the user
order_id_input = input("Enter Order ID: ")
check_warranty(order_id_input)


The warranty for Order ID ORD1005 is still valid.
Expiry Date: 2024-08-18
